In [1]:
import re
import torch
import pandas as pd
from konlpy.tag import Okt
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

In [2]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020.csv"
df = pd.read_csv(data_path)

In [3]:
# 중요 키워드 리스트 정의
keywords = ["원화", "원/달러"]

# "달러", "환율", "외환", "외환시장", "통화", "금리", "환전", "외화", "환변동"

# 중요 단어 포함 여부 
def contains_keyword(text):
    if pd.isna(text): 
        return False
    return any(keyword in text for keyword in keywords)

# 필터링 적용 
df = df[df["title"].apply(contains_keyword) | df["content"].apply(contains_keyword)]

# 정규표현식 패턴 정의
patterns_content = [
    r"\[.*?\]",  # 대괄호 안의 내용 및 대괄호 자체 제거
    r"\(.*?\)",  # 소괄호 안의 내용 및 소괄호 자체 제거
    r"[^ ]+@[^ ]+\.[a-z]{2,3}",  # 이메일 주소 제거
    r"▶.*?$",  # ▶로 시작하는 광고성 문구 제거
    r".*?=",  # "=" 이전 모든 내용 삭제 및 "=" 삭제
]

patterns_title = [
    r"\[.*?\]",  # 대괄호 안의 내용 및 대괄호 자체 제거
    r"\(.*?\)",  # 소괄호 안의 내용 및 소괄호 자체 제거
]

# 데이터 정제
for pattern in patterns_content:
    df["content"] = df["content"].apply(lambda x: re.sub(pattern, "", x).strip())

for pattern in patterns_title:
    df["title"] = df["title"].apply(lambda x: re.sub(pattern, "", x).strip())
    
# 중복제거 
df = df.drop_duplicates(subset=["title", "content"], keep="first")

# 마지막 문장을 추출하는 함수
def remove_last_sentence_if_no_period(text):
    if pd.isna(text):  
        return text
    
    # 문장을 문장부호 기준으로 분리 (". ", "! ", "? " 사용)
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    
    # 문장이 없는 경우 그대로 반환
    if not sentences:
        return text

    # 마지막 문장이 "."으로 끝나지 않으면 삭제
    if not sentences[-1].endswith("."):
        sentences.pop()  # 마지막 문장 제거

    return " ".join(sentences)  # 다시 문장 합치기

# content에만 적용
df["content"] = df["content"].apply(remove_last_sentence_if_no_period)

print(f"{len(df)}개의 뉴스가 남았습니다.")

1155개의 뉴스가 남았습니다.


In [7]:
# 결측치가 있는 행 제거
df = df.dropna(subset=['content'])
print(f"{len(df)}개의 뉴스가 남았습니다.")

1155개의 뉴스가 남았습니다.


In [8]:
# content 결측치면 title로 대체
df["content"] = df.apply(lambda row: row["title"] if pd.isna(row["content"]) or row["content"].strip() == "" else row["content"], axis=1)

In [9]:
df

,date,title,url,content
2,20200101,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치",https://n.news.naver.com/mnews/article/001/001...,외환당국이 지난 3분기 시장안정을 위해 외환시장에서 28억7천만달러를 순매도했다고 ...
6,20200101,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율은 롤러코스터를 타는 듯했다. 연중 고점과 저점 차이가 110원 ...
7,20200101,"외환시장 올해 마지막 거래일…원/달러 환율 1,156원에 마감",https://n.news.naver.com/mnews/article/001/001...,"외환시장 올해 마지막 거래일…원/달러 환율 1,156원에 마감"
8,20200101,"위험선호 분위기 속 원/달러 환율 1,150원대로 하락 출발",https://n.news.naver.com/mnews/article/001/001...,30일 오전 원/달러 환율이 하락한 채로 출발했다.이날 서울 외환시장에서 원/달러 ...
11,20200101,美中무역분쟁·브렉시트 여전히 대외리스크…美대선도 관건,https://n.news.naver.com/mnews/article/001/001...,연합뉴스 내년 경제전망 설문에 참여한 경제기관장들. 왼쪽부터 최정표 한국개발연구원 ...
...,...,...,...,...
7116,20201229,"원/달러 환율 4.6원 내린 1,092.1원",https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 4.6원 내린 1,092.1원"
7117,20201229,"""북한 당국, 국내 시장서 외화 사용 금지""<아사히>",https://n.news.naver.com/mnews/article/001/001...,북한 당국이 최근 국내 시장에서 외화 사용을 금지하기 시작했다고 아사히신문이 한국 ...
7119,20201229,"원/달러 환율 0.8원 오른 1,097.5원",https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 0.8원 오른 1,097.5원"
7136,20201230,"원/달러 환율 5.8원 내린 1,086.3원",https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 5.8원 내린 1,086.3원"


In [10]:
save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020_prep_3.csv"
df.to_csv(save_path, index=False, encoding="utf-8-sig")

In [ ]:
# KoBart 요약 -> 이상하게 나옴;;

In [5]:
# KoBART 모델 (SKT의 KoBART GitHub: https://github.com/SKT-AI/KoBART)
model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# GPU 사용 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 요약 함수 정의
def kobart_summarize(text, max_input_length=1024, max_target_length=128):

    # 인코딩
    inputs = tokenizer.encode_plus(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length,
        add_special_tokens=True
    )
    
    # GPU로 옮기기 
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # 문장 생성 
    summary_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_beams=4,          # 빔 서치 개수
        max_length=max_target_length,
        early_stopping=True,
        no_repeat_ngram_size=3  # 반복 방지
    )

    # 토크나이저로 디코딩
    summarized_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summarized_text.strip()

# CSV 파일 불러오기
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020_prep_1.csv"
df = pd.read_csv(data_path)

# summarize_content 생성
df["summarize_content"] = df["content"].apply(lambda x: kobart_summarize(str(x)))

# 결과 확인
df["summarize_content"].head(3)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


0                                                     
1    가가 등했다.다.국내 성장세가 낮아진 가운데 미·중 무역분쟁, 일본 수출규제, 홍콩...
2    n n 높 n n n 많이 n nn n n 등 n n랑 n n들 n n 순 n n ...
Name: summarize_content, dtype: object

In [6]:
save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\naver_finance_news_2020_prep_2.csv"
df.to_csv(save_path, index=False, encoding="utf-8-sig")